In [2]:
import pymysql
import pandas as pd
import time
import numpy as np
import os
import re
from datetime import datetime
from sqlalchemy import create_engine
import cryptography

In [3]:
# 時間欄位的清整方法
def format_time(number):
    float_number = float(number)
    int_number = int(float_number) 
    hours = int_number // 10000
    minutes = (int_number // 100) % 100
    seconds = int_number % 100
    return "{:02d}:{:02d}:{:02d}".format(hours, minutes, seconds)

# 日期欄位的清整方法
def format_date(number):
    from datetime import datetime
    date_string = number.strip() # 將整數轉換為字串
    date_data = datetime.strptime(date_string, '%Y%m%d').date()
    return date_data

# 死亡人數切分方法
def spilt_dead_num(Casualties):
    match = re.search(dead_num, Casualties)
    return match.group(1)

# 受傷人數切分方法
def spilt_injuried_num(Casualties):
    match = re.search(injuried_num, Casualties)
    return match.group(1)

# 縣市名稱切分方法
def spilt_city(address):
    return address[0:3]

## 讀單一檔案

In [4]:

#建立與 MySQL 資料庫的連線

host = 'ip address'
user = 'your account'
password = 'your password'
database = 'your DB name'
port = 3306


# 建立mysql連線
conn = pymysql.connect(host=host, user=user, password=password, database=database, port=port)

# 指定要寫入的資料庫表名稱
table_name = 'ACCIDENT'
dead_num = r"死亡(\d+)"
injuried_num = r"受傷(\d+)"

source_path = "../RAW/全國交通事故資料/BIG_ALL.csv"


# 讀取csv檔並將發生年度、發生月份欄位的資料刪除
df = pd.read_csv(source_path,encoding='UTF-8-sig', header=0, index_col=False).drop(["Unnamed: 0","發生年度","發生月份"], axis=1)

# 將最下面兩行說明刪除
df = df.dropna(axis=0,subset=['發生日期'])

# 時間欄位清整
df['發生時間'] = df['發生時間'].astype(str).str.replace(':', '')
df['發生時間'] = df['發生時間'].apply(lambda x: x.split('.')[0])
df['發生時間'] = df['發生時間'].apply(lambda x: x.zfill(6))
df['發生時間'] = df['發生時間'].str[0:2]
df['發生時間'] = df['發生時間'].astype(int)+1
df['發生時間'] = df['發生時間'].apply(lambda x: f"{x}:00:00")

# 日期欄位清整
df['發生日期'] = df['發生日期'].astype(str).str.replace('-', '')
df['發生日期'] = df['發生日期'].apply(lambda x: x.split('.')[0])
df['發生日期'] = df['發生日期'].apply(lambda x: format_date(x))

# 創建一个空的列
new_column = pd.Series(dtype='int')

# 在指定位置插入空的列
df.insert(31, '死亡人數', new_column)
df.insert(32, '受傷人數', new_column)
df.insert(4, '縣市名稱', new_column)

# 將死亡人數與受傷人數切分
df['死亡人數'] = df['死亡受傷人數'].apply(lambda x: spilt_dead_num(x))
df['受傷人數'] = df['死亡受傷人數'].apply(lambda x: spilt_injuried_num(x))
# 將死亡受傷人數一整列的資料刪除
df = df.drop(["死亡受傷人數"], axis=1)

# 切分縣市名稱
df['縣市名稱'] = df['發生地點'].apply(lambda x: spilt_city(x))
#print(df)

# 把發生地點的特殊符號拿掉
special_characters = ['[!@#$%^&*]', '[+=~`{}[\]|:;"\',<.>/?]']

# special_characters = ['[!@#$%^&*()]', '[+=~`{}[\]|:;"\',<.>/?]']
regex_pattern = '|'.join(special_characters)
df['發生地點'] = df['發生地點'].replace(regex_pattern, '', regex=True)
#print(df)

    # 一行一行寫入資料庫
for row in df.itertuples(index=False):
    values = ', '.join([f"'{value}'" if pd.notnull(value) else 'NULL' for value in row])
    query = f"INSERT INTO {table_name} VALUES ({values})"
    try:
        with conn.cursor() as cursor:

            cursor.execute(query)

            # 將上面for迴圈的內容commit到資料庫    


    except pymysql.MySQLError as e:
        print(f"row: {row}: {str(e)}\n")

conn.commit()
print(f"Finished processing row {row}")
#     finally:
#         # 這部分的代碼無論是否有錯誤都會執行
#         conn.commit()   # 最後剩餘的commit
#         print(f"Finished processing row {row}")

# 關閉資料庫連線
conn.close()

row: Pandas(發生日期=datetime.date(2020, 5, 31), 發生時間='16:00:00', 事故類別名稱='A2', 處理單位名稱警局層='桃園市政府警察局', 縣市名稱='桃園市', 發生地點='桃園市中壢區忠孝路157號對面(內壢派出所)\\', 天候名稱='晴', 光線名稱='日間自然光線', _8='市區道路', _9=40.0, 道路型態大類別名稱='交岔路', 道路型態子類別名稱='三岔路', 事故位置大類別名稱='路段', 事故位置子類別名稱='一般車道(未劃分快慢車道)', _14='柏油', _15='乾燥', _16='無缺陷', _17='無障礙物', _18='良好', _19='良好', _20='無號誌', _21='無號誌', _22='行車分向線', _23='附標記', _24='車道線(附標記)', _25='未繪設快慢車道分隔線', _26='有', 事故類型及型態大類別名稱='車與車', 事故類型及型態子類別名稱='同向擦撞', _29='駕駛人', _30='未注意車前狀態', 死亡人數='0', 受傷人數='1', 當事者順位=1.0, _34='機車', _35='普通重型', _36='男', 當事者事故發生時年齡=18.0, 保護裝備名稱='戴安全帽或繫安全帶(使用幼童安全椅)', 行動電話或電腦或其他相類功能裝置名稱='未使用', 當事者行動狀態大類別名稱='車的狀態', 當事者行動狀態子類別名稱='向前直行中', _42='機車', _43='前車頭', _44=nan, _45=nan, _46='駕駛人', _47='未注意車前狀態', _48='否', 經度=121.254862, 緯度=24.976589): (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '晴', '日間自然光線', '市區道路', '40.0', '交岔路', '三岔路', 'è' at line 1")

row: Pandas(發生日期=datetime.date(2

## 讀資料夾內

In [ ]:

#建立與 MySQL 資料庫的連線
host = 'ip address'
user = 'your account'
password = 'your password'
database = 'your DB name'
port = 3306

# 建立mysql連線
conn = pymysql.connect(host=host, user=user, password=password, database=database, port=port)

# 指定要寫入的資料庫表名稱
table_name = 'ACCIDENT'
dead_num = r"死亡(\d+)"
injuried_num = r"受傷(\d+)"

source_path = "../RAW/全國交通事故資料/ALL"



# 讀取source_path裡的所有檔案
for filename in os.listdir(source_path):

    # 讀取csv檔並將發生年度、發生月份欄位的資料刪除
    df = pd.read_csv("{}/{}".format(source_path,filename),encoding='UTF-8-sig', header=0, index_col=False).drop(["Unnamed: 0","發生年度","發生月份"], axis=1)

    # 將最下面兩行說明刪除
    df = df.dropna(axis=0,subset=['發生日期'])

    # 時間欄位清整
    df['發生時間'] = df['發生時間'].astype(str).str.replace(':', '')
    df['發生時間'] = df['發生時間'].apply(lambda x: x.split('.')[0])
    df['發生時間'] = df['發生時間'].apply(lambda x: x.zfill(6))
    df['發生時間'] = df['發生時間'].str[0:2]
    df['發生時間'] = df['發生時間'].astype(int)+1
    df['發生時間'] = df['發生時間'].apply(lambda x: f"{x}:00:00")

    # 日期欄位清整
    df['發生日期'] = df['發生日期'].astype(str).str.replace('-', '')
    df['發生日期'] = df['發生日期'].apply(lambda x: x.split('.')[0])
    df['發生日期'] = df['發生日期'].apply(lambda x: format_date(x))

    # 創建一个空的列
    new_column = pd.Series(dtype='int')

    # 在指定位置插入空的列
    df.insert(31, '死亡人數', new_column)
    df.insert(32, '受傷人數', new_column)
    df.insert(4, '縣市名稱', new_column)

    # 將死亡人數與受傷人數切分
    df['死亡人數'] = df['死亡受傷人數'].apply(lambda x: spilt_dead_num(x))
    df['受傷人數'] = df['死亡受傷人數'].apply(lambda x: spilt_injuried_num(x))
    # 將死亡受傷人數一整列的資料刪除
    df = df.drop(["死亡受傷人數"], axis=1)

    # 切分縣市名稱
    df['縣市名稱'] = df['發生地點'].apply(lambda x: spilt_city(x))
    #print(df)
    
    # 把發生地點的特殊符號拿掉
    special_characters = ['[!@#$%^&*()]', '[+=~`{}[\]|:;"\',<.>/?]']
    regex_pattern = '|'.join(special_characters)
    df['發生地點'] = df['發生地點'].replace(regex_pattern, '', regex=True)
    #print(df)
    
        # 一行一行寫入資料庫
    for row in df.itertuples(index=False):
        values = ', '.join([f"'{value}'" if pd.notnull(value) else 'NULL' for value in row])
        query = f"INSERT INTO {table_name} VALUES ({values})"
        try:
            with conn.cursor() as cursor:

                cursor.execute(query)

                # 將上面for迴圈的內容commit到資料庫    
            

        except pymysql.MySQLError as e:
            print(f"Error inserting file:{filename}, row: {row}: {str(e)}\n")

    conn.commit()
    print(f"Finished processing row {row}")
#     finally:
#         # 這部分的代碼無論是否有錯誤都會執行
#         conn.commit()   # 最後剩餘的commit
#         print(f"Finished processing row {row}")

# 關閉資料庫連線
conn.close()

        